In [6]:
import pandas as pd
import numpy as np

import os
import random

from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib

import warnings
warnings.filterwarnings(action='ignore')

random.seed(123)
np.random.seed(123)

pd.set_option('display.max_columns', 200)

root_dir = '/'.join(os.path.realpath(__name__).split('/')[:-2])
input_dir = root_dir + '/input_data'
persist_dir = root_dir + '/persist'

def save_check_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print('>> {} << directory created.'.format(dir_path.split('/')[-1]))
        
    else:
        print(dir_path)
        print('>> {} << directory already exists.'.format(dir_path.split('/')[-1]))

save_check_dir(persist_dir)

/Users/300029144/Documents/kaggle/home_credit_default/home-credit-default/persist
>> persist << directory already exists.


In [2]:
credit_card_path = input_dir + '/credit_card_balance.csv'
credit_card = pd.read_csv(credit_card_path)

# Data Cleaning

In [7]:
credit_card['AMT_DRAWINGS_ATM_CURRENT'][credit_card['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
credit_card['AMT_DRAWINGS_CURRENT'][credit_card['AMT_DRAWINGS_CURRENT'] < 0] = np.nan

In [4]:
credit_card.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [10]:
def _static_features(credit_card, **kwargs):
    credit_card['number_of_installments'] = credit_card.groupby(
        by=['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].agg('max').reset_index()[
        'CNT_INSTALMENT_MATURE_CUM']

    credit_card['credit_card_max_loading_of_credit_limit'] = credit_card.groupby(
        by=['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']).apply(
        lambda x: x.AMT_BALANCE.max() / x.AMT_CREDIT_LIMIT_ACTUAL.max()).reset_index()[0]

    features = pd.DataFrame({'SK_ID_CURR': credit_card['SK_ID_CURR'].unique()})

    groupby = credit_card.groupby(by=['SK_ID_CURR'])

    g = groupby['SK_ID_PREV'].agg('nunique').reset_index()
    g.rename(index=str, columns={'SK_ID_PREV': 'credit_card_number_of_loans'}, inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    g = groupby['SK_DPD'].agg('mean').reset_index()
    g.rename(index=str, columns={'SK_DPD': 'credit_card_average_of_days_past_due'}, inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    g = groupby['AMT_DRAWINGS_ATM_CURRENT'].agg('sum').reset_index()
    g.rename(index=str, columns={'AMT_DRAWINGS_ATM_CURRENT': 'credit_card_drawings_atm'}, inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    g = groupby['AMT_DRAWINGS_CURRENT'].agg('sum').reset_index()
    g.rename(index=str, columns={'AMT_DRAWINGS_CURRENT': 'credit_card_drawings_total'}, inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    g = groupby['number_of_installments'].agg('sum').reset_index()
    g.rename(index=str, columns={'number_of_installments': 'credit_card_total_installments'}, inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    g = groupby['credit_card_max_loading_of_credit_limit'].agg('mean').reset_index()
    g.rename(index=str,
             columns={'credit_card_max_loading_of_credit_limit': 'credit_card_avg_loading_of_credit_limit'},
             inplace=True)
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    features['credit_card_cash_card_ratio'] = features['credit_card_drawings_atm'] / features[
        'credit_card_drawings_total']

    features['credit_card_installments_per_loan'] = (
        features['credit_card_total_installments'] / features['credit_card_number_of_loans'])

    return features

def _dynamic_features(credit_card, **kwargs):
    features = pd.DataFrame({'SK_ID_CURR': credit_card['SK_ID_CURR'].unique()})

    credit_card_sorted = credit_card.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE'])

    groupby = credit_card_sorted.groupby(by=['SK_ID_CURR'])
    credit_card_sorted['credit_card_monthly_diff'] = groupby['AMT_BALANCE'].diff()
    
    groupby = credit_card_sorted.groupby(by=['SK_ID_CURR'])
    g = groupby['credit_card_monthly_diff'].agg('mean').reset_index()
    features = features.merge(g, on=['SK_ID_CURR'], how='left')

    return features


static_features = _static_features(credit_card)
dynamic_features = _dynamic_features(credit_card)

features =  pd.merge(static_features,
                     dynamic_features,
                     on=['SK_ID_CURR'],
                     validate='one_to_one')

In [12]:
CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_BALANCE',
                   'AMT_CREDIT_LIMIT_ACTUAL',
                   'AMT_DRAWINGS_ATM_CURRENT',
                   'AMT_DRAWINGS_CURRENT',
                   'AMT_DRAWINGS_OTHER_CURRENT',
                   'AMT_DRAWINGS_POS_CURRENT',
                   'AMT_PAYMENT_CURRENT',
                   'CNT_DRAWINGS_ATM_CURRENT',
                   'CNT_DRAWINGS_CURRENT',
                   'CNT_DRAWINGS_OTHER_CURRENT',
                   'CNT_INSTALMENT_MATURE_CUM',
                   'MONTHS_BALANCE',
                   'SK_DPD',
                   'SK_DPD_DEF'
                   ]:
        CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES.append((select, agg))
CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES)]

In [13]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES):
    group_object = credit_card.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        features = features.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [15]:
len(features.columns)

80

In [16]:
credit_persist_path = persist_dir + '/credit_card.pkl'
joblib.dump(features, credit_persist_path)

['/Users/300029144/Documents/kaggle/home_credit_default/home-credit-default/persist/credit_card.pkl']